<a href="https://colab.research.google.com/github/mahmoudiwalid/Sereniiti_final_model/blob/main/Sereniiti_Final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# download  the english language pipeline of spacy
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=502f13a3130ffc2c2af41b3e258905a36a822581e14d71b7c1de9b891f13922c
  Stored in directory: /tmp/pip-ephem-wheel-cache-0p0q7ybg/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [1]:
import spacy
import en_core_web_lg
nlp = spacy.load('en_core_web_lg')

In [31]:
import numpy as np

In [2]:
import pandas as pd

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [72]:
from sklearn.metrics import mean_squared_error

In [63]:
import pickle

In [15]:
from spacy import displacy

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Sereniiti/sereniiti-sentences1.csv', encoding= 'unicode_escape')
df

,Unnamed: 0,id,en,fr,rating,tips,tips_groups,steps,fivefold,step_facts,step_emotions,step_needs,step_proposals,tip_morality,tip_positivebias,tip_interpretation,tip_comparison,tip_labelling,tip_generalisation,tip_duality,tip_reproach,tip_deserves,tip_denial,tip_sacrifice,tip_unavailable,tip_unsolicited,tip_provocation,tip_fact,tip_selfempathy,tip_empathy,tip_dialogue,tip_clarity,tip_activity,tip_demand,tip_person,tip_who,tip_location,tip_where,tip_time,tip_when,tip_action,tip_what,tip_objective,tip_why,tip_thismany,tip_howmany,tip_positive,tip_negative,tip_realistic,tip_unrealistic,tip_feeling,tip_need,group_empathy,group_negative,group_feeling,group_fault,group_unsolicited,group_need,group_realistic,group_precise,group_unprecise,group_unrealistic,group_provocation,group_positivebias,group_judgemental,group_demanding,group_fact,group_positive
0,1,1,"60,000 sperms, and you won?","60 000 spermatozoÃ¯des, et tu as gagnÃ© ?",0,"comparison,provocation","provocation,judgemental",facts,3.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
1,2,2,A 2014 study examined the effects of combined ...,Une Ã©tude de 2014 a examinÃ© les effets d'une...,4,fact,fact,facts,5.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,3,3,A big guy like you needs to stand up for himself!,Un grand gars comme toi doit savoir se dÃ©fendre!,1,"labelling,reproach","judgemental,fault",facts,4.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,4,4,"A cheetah, the fastest land animal, can run 70...","Un guÃ©pard, l'animal terrestre le plus rapide...",4,fact,fact,facts,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,5,5,A combination of words that makes sense is cal...,Une combinaison de mots qui a un sens complet ...,4,fact,fact,facts,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11296,11297,11295,Some of the people who come to us for food are...,Certaines des personnes qui viennent nous dema...,5,need,need,"facts,emotions,needs",1.0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
11297,11298,11296,Would you mind telling me how you feel when yo...,Pourriez-vous me dire comment vous vous sentez...,5,dialogue,empathy,"facts,emotions,needs",2.0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11298,11299,11297,I feel frustrated,je me sens frustrÃ©e,4,feeling,feeling,emotions,3.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
11299,11300,11298,"Each time, you take the day before to start re...","A chaque fois, tu t'y prend la veille pour com...",1,interpretation,judgemental,facts,4.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [4]:
#store only english language text
df = df.drop('fr',1)
df = df.drop('fivefold',1)
#remove missing value NaN 
#df = df.dropna()
df

,Unnamed: 0,id,en,rating,tips,tips_groups,steps,step_facts,step_emotions,step_needs,step_proposals,tip_morality,tip_positivebias,tip_interpretation,tip_comparison,tip_labelling,tip_generalisation,tip_duality,tip_reproach,tip_deserves,tip_denial,tip_sacrifice,tip_unavailable,tip_unsolicited,tip_provocation,tip_fact,tip_selfempathy,tip_empathy,tip_dialogue,tip_clarity,tip_activity,tip_demand,tip_person,tip_who,tip_location,tip_where,tip_time,tip_when,tip_action,tip_what,tip_objective,tip_why,tip_thismany,tip_howmany,tip_positive,tip_negative,tip_realistic,tip_unrealistic,tip_feeling,tip_need,group_empathy,group_negative,group_feeling,group_fault,group_unsolicited,group_need,group_realistic,group_precise,group_unprecise,group_unrealistic,group_provocation,group_positivebias,group_judgemental,group_demanding,group_fact,group_positive
0,1,1,"60,000 sperms, and you won?",0,"comparison,provocation","provocation,judgemental",facts,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
1,2,2,A 2014 study examined the effects of combined ...,4,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,3,3,A big guy like you needs to stand up for himself!,1,"labelling,reproach","judgemental,fault",facts,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,4,4,"A cheetah, the fastest land animal, can run 70...",4,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,5,5,A combination of words that makes sense is cal...,4,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11296,11297,11295,Some of the people who come to us for food are...,5,need,need,"facts,emotions,needs",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
11297,11298,11296,Would you mind telling me how you feel when yo...,5,dialogue,empathy,"facts,emotions,needs",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11298,11299,11297,I feel frustrated,4,feeling,feeling,emotions,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
11299,11300,11298,"Each time, you take the day before to start re...",1,interpretation,judgemental,facts,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [5]:
import re
import string
def get_clean(text):

  '''
  This is a cleaning text function
  To remove the punctuations
  will consider only alphabets and numerics
  will replace newline with space
  will convert to lower case
  will split and join the words
  '''
  text=re.sub('<.*?>', ' ', text)  
  #text = text.translate(str.maketrans(' ',' ',string.punctuation))
  text = re.sub('[^a-zA-Z]',' ',text)  
  text = re.sub("\n"," ",text)
  text = text.lower()
  text=' '.join(text.split())
  return text

In [6]:
#apply the function "get clean" to the data 
df['en'] = df['en'].apply(get_clean)

In [7]:
# labelling the data into 2 categories Bad {0, 1, 2 } --> 0 / Good {3, 4, 5, 6} --> 1
label = {0:0 , 1:0 ,2:0, 3:1 , 4:1 , 5:1 , 6:1}
df['label']=df['rating'].map(label)
df

,Unnamed: 0,id,en,rating,tips,tips_groups,steps,step_facts,step_emotions,step_needs,step_proposals,tip_morality,tip_positivebias,tip_interpretation,tip_comparison,tip_labelling,tip_generalisation,tip_duality,tip_reproach,tip_deserves,tip_denial,tip_sacrifice,tip_unavailable,tip_unsolicited,tip_provocation,tip_fact,tip_selfempathy,tip_empathy,tip_dialogue,tip_clarity,tip_activity,tip_demand,tip_person,tip_who,tip_location,tip_where,tip_time,tip_when,tip_action,tip_what,tip_objective,tip_why,tip_thismany,tip_howmany,tip_positive,tip_negative,tip_realistic,tip_unrealistic,tip_feeling,tip_need,group_empathy,group_negative,group_feeling,group_fault,group_unsolicited,group_need,group_realistic,group_precise,group_unprecise,group_unrealistic,group_provocation,group_positivebias,group_judgemental,group_demanding,group_fact,group_positive,label
0,1,1,sperms and you won,0,"comparison,provocation","provocation,judgemental",facts,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
1,2,2,a study examined the effects of combined nvc a...,4,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,3,3,a big guy like you needs to stand up for himself,1,"labelling,reproach","judgemental,fault",facts,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,4,a cheetah the fastest land animal can run mile...,4,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
4,5,5,a combination of words that makes sense is cal...,4,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11296,11297,11295,some of the people who come to us for food are...,5,need,need,"facts,emotions,needs",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
11297,11298,11296,would you mind telling me how you feel when yo...,5,dialogue,empathy,"facts,emotions,needs",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
11298,11299,11297,i feel frustrated,4,feeling,feeling,emotions,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
11299,11300,11298,each time you take the day before to start rev...,1,interpretation,judgemental,facts,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [8]:
#The newly created column "rating" is added at the tail of the table
cols = df.columns.tolist()
cols

['Unnamed: 0',
 'id',
 'en',
 'rating',
 'tips',
 'tips_groups',
 'steps',
 'step_facts',
 'step_emotions',
 'step_needs',
 'step_proposals',
 'tip_morality',
 'tip_positivebias',
 'tip_interpretation',
 'tip_comparison',
 'tip_labelling',
 'tip_generalisation',
 'tip_duality',
 'tip_reproach',
 'tip_deserves',
 'tip_denial',
 'tip_sacrifice',
 'tip_unavailable',
 'tip_unsolicited',
 'tip_provocation',
 'tip_fact',
 'tip_selfempathy',
 'tip_empathy',
 'tip_dialogue',
 'tip_clarity',
 'tip_activity',
 'tip_demand',
 'tip_person',
 'tip_who',
 'tip_location',
 'tip_where',
 'tip_time',
 'tip_when',
 'tip_action',
 'tip_what',
 'tip_objective',
 'tip_why',
 'tip_thismany',
 'tip_howmany',
 'tip_positive',
 'tip_negative',
 'tip_realistic',
 'tip_unrealistic',
 'tip_feeling',
 'tip_need',
 'group_empathy',
 'group_negative',
 'group_feeling',
 'group_fault',
 'group_unsolicited',
 'group_need',
 'group_realistic',
 'group_precise',
 'group_unprecise',
 'group_unrealistic',
 'group_provocat

In [9]:
#We are going to bring it in the 5th position
df= df[['Unnamed: 0','id','en','rating','label','tips','tips_groups','steps','step_facts','step_emotions','step_needs','step_proposals','tip_morality','tip_positivebias','tip_interpretation','tip_comparison','tip_labelling','tip_generalisation','tip_duality','tip_reproach','tip_deserves','tip_denial','tip_sacrifice','tip_unavailable','tip_unsolicited','tip_provocation','tip_fact','tip_selfempathy','tip_empathy','tip_dialogue','tip_clarity','tip_activity','tip_demand','tip_person','tip_who','tip_location','tip_where','tip_time','tip_when','tip_action','tip_what','tip_objective','tip_why','tip_thismany','tip_howmany','tip_positive','tip_negative','tip_realistic','tip_unrealistic','tip_feeling','tip_need','group_empathy','group_negative','group_feeling','group_fault','group_unsolicited','group_need','group_realistic','group_precise','group_unprecise','group_unrealistic','group_provocation','group_positivebias','group_judgemental','group_demanding','group_fact','group_positive']]

In [10]:
df

,Unnamed: 0,id,en,rating,label,tips,tips_groups,steps,step_facts,step_emotions,step_needs,step_proposals,tip_morality,tip_positivebias,tip_interpretation,tip_comparison,tip_labelling,tip_generalisation,tip_duality,tip_reproach,tip_deserves,tip_denial,tip_sacrifice,tip_unavailable,tip_unsolicited,tip_provocation,tip_fact,tip_selfempathy,tip_empathy,tip_dialogue,tip_clarity,tip_activity,tip_demand,tip_person,tip_who,tip_location,tip_where,tip_time,tip_when,tip_action,tip_what,tip_objective,tip_why,tip_thismany,tip_howmany,tip_positive,tip_negative,tip_realistic,tip_unrealistic,tip_feeling,tip_need,group_empathy,group_negative,group_feeling,group_fault,group_unsolicited,group_need,group_realistic,group_precise,group_unprecise,group_unrealistic,group_provocation,group_positivebias,group_judgemental,group_demanding,group_fact,group_positive
0,1,1,sperms and you won,0,0,"comparison,provocation","provocation,judgemental",facts,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
1,2,2,a study examined the effects of combined nvc a...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,3,3,a big guy like you needs to stand up for himself,1,0,"labelling,reproach","judgemental,fault",facts,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,4,4,a cheetah the fastest land animal can run mile...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,5,5,a combination of words that makes sense is cal...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11296,11297,11295,some of the people who come to us for food are...,5,1,need,need,"facts,emotions,needs",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
11297,11298,11296,would you mind telling me how you feel when yo...,5,1,dialogue,empathy,"facts,emotions,needs",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11298,11299,11297,i feel frustrated,4,1,feeling,feeling,emotions,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
11299,11300,11298,each time you take the day before to start rev...,1,0,interpretation,judgemental,facts,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


#Tokonization

In [11]:
def nlp_pipeline (x):
  '''
  define a pipeline function to tokenize the data
  '''
  doc = nlp(x)
  return doc

In [12]:
# apply the pipeline function for the hole dataset and store it in new column called 'pipeline'
df['pipeline'] = df['en'].apply(nlp_pipeline)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


###Named entity recognition

In [13]:
#for ent in doc.ents:
  #print(ent.text, ent.label_)

#Sentence segmentation

In [14]:
def get_sentence (x):
  '''
  Define function that dectect the sentence boudary:
  '''
  doc = nlp(x)
  for sent in doc.sents:
    print(sent)
  return sent

#Part of speech tagging
## Is the process of marking up a word in a text (corpus) as corresponding to a particular part of speech, based on both its definition and its context.

In [16]:
doc1=nlp_pipeline("a big guy like you needs to stand up for himself")
displacy.render(doc1, style = 'dep', jupyter=True)

#Chunking: 
The chunking process is used of extracting noun phrases from the text. Think of noun phrase (or noun chunks in Spacy) as the noun plus the words describing the noun.

In [17]:
#to illustrate how chunking is important take the example of the word "combined": ot was tokonized as a verb
#but here it is assigned to its real function in sentnece ==> object of preposition
doc3=nlp_pipeline("a study examined the effects of combined nvc and mindfulness training on male inmates of the monroe correctional complex in monroe washington")
print(doc3, ":")
print("**********")
for chunk in doc3.noun_chunks:
    print(chunk.text,  "|", spacy.explain(chunk.root.dep_))

a study examined the effects of combined nvc and mindfulness training on male inmates of the monroe correctional complex in monroe washington :
**********
a study | nominal subject
the effects | direct object
combined nvc | object of preposition
mindfulness training | conjunct
male inmates | object of preposition
the monroe correctional complex | object of preposition
monroe washington | object of preposition


In [18]:
def get_subject_phrase(doc):
  '''
  function that extrat the subject of phrase:
  '''
  for token in doc:
    if ("subj" in token.dep_):
      subtree = list(token.subtree)
      start = subtree[0].i
      end = subtree[-1].i + 1
      return doc[start:end]

In [19]:
def get_object_phrase(doc):
  '''
  function that extract object of phrase:
  '''
  for token in doc:
    if ("dobj" in token.dep_):
      subtree = list(token.subtree)
      start = subtree[0].i
      end = subtree[-1].i + 1
      return doc[start:end]

#Function that detects sentence boundary and split the sentence to its verbal untis: Subject - Verb - Object

In [20]:
def verbal_unit (x):
  '''
  Build a function that return subject verb and object
  '''
  ls=[]
  doc=nlp(x)
  subject_phrase = get_subject_phrase(doc)
  subj_phrase = str(subject_phrase)
  verb_phrase0=x.replace(subj_phrase,'')
  object_phrase = get_object_phrase(doc)
  obj_phrase = str(object_phrase)
  verb_phrase1=verb_phrase0.replace(obj_phrase,'')
  ls=[subj_phrase,verb_phrase1,obj_phrase]
  return(ls)

In [21]:
# Example I took a comlpex and long sentence from the data set
# verbal_unit("a study examined the effects of combined nvc and mindfulness training on male inmates of the monroe correctional complex in monroe washington")

In [21]:
df['Verbal_units']=df['en'].apply(verbal_unit)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,id,en,rating,label,tips,tips_groups,steps,step_facts,step_emotions,step_needs,step_proposals,tip_morality,tip_positivebias,tip_interpretation,tip_comparison,tip_labelling,tip_generalisation,tip_duality,tip_reproach,tip_deserves,tip_denial,tip_sacrifice,tip_unavailable,tip_unsolicited,tip_provocation,tip_fact,tip_selfempathy,tip_empathy,tip_dialogue,tip_clarity,tip_activity,tip_demand,tip_person,tip_who,tip_location,tip_where,tip_time,tip_when,tip_action,tip_what,tip_objective,tip_why,tip_thismany,tip_howmany,tip_positive,tip_negative,tip_realistic,tip_unrealistic,tip_feeling,tip_need,group_empathy,group_negative,group_feeling,group_fault,group_unsolicited,group_need,group_realistic,group_precise,group_unprecise,group_unrealistic,group_provocation,group_positivebias,group_judgemental,group_demanding,group_fact,group_positive,pipeline,Verbal_units
0,1,1,sperms and you won,0,0,"comparison,provocation","provocation,judgemental",facts,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,"(sperms, and, you, won)","[you, sperms and won, None]"
1,2,2,a study examined the effects of combined nvc a...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, study, examined, the, effects, of, combine...","[a study, examined , the effects of combined ..."
2,3,3,a big guy like you needs to stand up for himself,1,0,"labelling,reproach","judgemental,fault",facts,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,"(a, big, guy, like, you, needs, to, stand, up,...","[you, a big guy like needs to stand up for hi..."
3,4,4,a cheetah the fastest land animal can run mile...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, cheetah, the, fastest, land, animal, can, ...","[a cheetah, the fastest land animal can run ,..."
4,5,5,a combination of words that makes sense is cal...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, combination, of, words, that, makes, sense...","[a combination of words that makes sense, is ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11296,11297,11295,some of the people who come to us for food are...,5,1,need,need,"facts,emotions,needs",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,"(some, of, the, people, who, come, to, us, for...","[some of the people who come to us for food, ..."
11297,11298,11296,would you mind telling me how you feel when yo...,5,1,dialogue,empathy,"facts,emotions,needs",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(would, you, mind, telling, me, how, you, feel...","[you, would mind telling how feel when see..."
11298,11299,11297,i feel frustrated,4,1,feeling,feeling,emotions,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"(i, feel, frustrated)","[i, feel frustrated, None]"
11299,11300,11298,each time you take the day before to start rev...,1,0,interpretation,judgemental,facts,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,"(each, time, you, take, the, day, before, to, ...","[you, each time take before to start revisin..."


In [22]:
# Example I took a comlpex and long sentence from the data set/ the output is ['subject', 'verb', 'object']
verbal_unit("a study examined the effects of combined nvc and mindfulness training on male inmates of the monroe correctional complex in monroe washington")

['a study',
 ' examined ',
 'the effects of combined nvc and mindfulness training on male inmates of the monroe correctional complex in monroe washington']

#Word embedding: getting a vector of each verbal unit: subject.vector / verb.vector / dobject.vector

In [23]:
#using the spacy build-in word embedding function "".vector" it is by default provides 300-dimensional vector
def get_vector(x):
  ls=[]
  doc=nlp_pipeline(x)
  subject_phrase = get_subject_phrase(doc)
  subject_vector= subject_phrase.vector
  subj_phrase = str(subject_phrase)
  verb_phrase0=x.replace(subj_phrase,'')
  object_phrase = get_object_phrase(doc)
  object_vector = object_phrase.vector
  obj_phrase = str(object_phrase)
  verb_phrase1=verb_phrase0.replace(obj_phrase,'')
  verbal_vector=nlp(verb_phrase1).vector
  ls=[subject_vector,verbal_vector,object_vector]
  return(ls)

In [24]:
#Example
#we will get an array of array: an array of 3 vectors (subject, verb an object)
get_vector("a study examined the effects of combined nvc and mindfulness training on male inmates of the monroe correctional complex in monroe washington")


[array([-9.95999947e-03, -3.55800018e-02, -1.00973554e-01,  1.37059987e-01,
        -1.28480002e-01, -1.52615502e-01, -1.89224005e-01, -1.94319993e-01,
         1.43717006e-01,  2.41195011e+00, -5.82584977e-01, -3.07171997e-02,
        -3.14900056e-02,  6.24991953e-05,  7.44000003e-02, -5.49164973e-02,
         4.58794981e-02,  1.84519994e+00, -1.49630502e-01, -2.29555503e-01,
         2.74055004e-01, -2.82698482e-01, -2.28483990e-01,  1.51138499e-01,
         6.69730008e-02,  1.80393994e-01,  3.02082002e-01,  8.95195007e-02,
         3.42545003e-01,  2.35975489e-01, -1.26488000e-01,  9.60140005e-02,
         2.94835001e-01, -2.08948001e-01,  2.77334988e-01, -8.73135030e-02,
         1.65795997e-01, -1.76463008e-01, -2.37224996e-01, -4.94239986e-01,
         1.89006492e-01,  3.31880003e-01,  1.56645000e-01, -2.34604999e-01,
         1.34899989e-02,  5.05299866e-03, -2.46205002e-01, -1.20655000e-01,
         2.96384990e-01, -6.44499809e-03, -4.68260020e-01,  2.80162483e-01,
         1.9

#Word embedding of the text in the dataset

In [25]:
#The string format is accepted for vectorisation
def list_to_string(x):
  char=str(x)
  return char

In [26]:
df['final_verbal_units']=df['Verbal_units'].apply(list_to_string)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,id,en,rating,label,tips,tips_groups,steps,step_facts,step_emotions,step_needs,step_proposals,tip_morality,tip_positivebias,tip_interpretation,tip_comparison,tip_labelling,tip_generalisation,tip_duality,tip_reproach,tip_deserves,tip_denial,tip_sacrifice,tip_unavailable,tip_unsolicited,tip_provocation,tip_fact,tip_selfempathy,tip_empathy,tip_dialogue,tip_clarity,tip_activity,tip_demand,tip_person,tip_who,tip_location,tip_where,tip_time,tip_when,tip_action,tip_what,tip_objective,tip_why,tip_thismany,tip_howmany,tip_positive,tip_negative,tip_realistic,tip_unrealistic,tip_feeling,tip_need,group_empathy,group_negative,group_feeling,group_fault,group_unsolicited,group_need,group_realistic,group_precise,group_unprecise,group_unrealistic,group_provocation,group_positivebias,group_judgemental,group_demanding,group_fact,group_positive,pipeline,Verbal_units,final_verbal_units
0,1,1,sperms and you won,0,0,"comparison,provocation","provocation,judgemental",facts,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,"(sperms, and, you, won)","[you, sperms and won, None]","['you', 'sperms and won', 'None']"
1,2,2,a study examined the effects of combined nvc a...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, study, examined, the, effects, of, combine...","[a study, examined , the effects of combined ...","['a study', ' examined ', 'the effects of comb..."
2,3,3,a big guy like you needs to stand up for himself,1,0,"labelling,reproach","judgemental,fault",facts,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,"(a, big, guy, like, you, needs, to, stand, up,...","[you, a big guy like needs to stand up for hi...","['you', 'a big guy like needs to stand up for..."
3,4,4,a cheetah the fastest land animal can run mile...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, cheetah, the, fastest, land, animal, can, ...","[a cheetah, the fastest land animal can run ,...","['a cheetah', ' the fastest land animal can ru..."
4,5,5,a combination of words that makes sense is cal...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, combination, of, words, that, makes, sense...","[a combination of words that makes sense, is ...","['a combination of words that makes sense', ' ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11296,11297,11295,some of the people who come to us for food are...,5,1,need,need,"facts,emotions,needs",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,"(some, of, the, people, who, come, to, us, for...","[some of the people who come to us for food, ...","['some of the people who come to us for food',..."
11297,11298,11296,would you mind telling me how you feel when yo...,5,1,dialogue,empathy,"facts,emotions,needs",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(would, you, mind, telling, me, how, you, feel...","[you, would mind telling how feel when see...","['you', 'would mind telling how feel when ..."
11298,11299,11297,i feel frustrated,4,1,feeling,feeling,emotions,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"(i, feel, frustrated)","[i, feel frustrated, None]","['i', ' feel frustrated', 'None']"
11299,11300,11298,each time you take the day before to start rev...,1,0,interp

In [27]:
def sent_vector(x):
  doc=nlp_pipeline(x)
  vect=doc.vector
  return(vect)

In [28]:
df['final_vector']=df['final_verbal_units'].apply(sent_vector)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
#list all columns for cleaning
cols = df.columns.tolist()
cols

['Unnamed: 0',
 'id',
 'en',
 'rating',
 'label',
 'tips',
 'tips_groups',
 'steps',
 'step_facts',
 'step_emotions',
 'step_needs',
 'step_proposals',
 'tip_morality',
 'tip_positivebias',
 'tip_interpretation',
 'tip_comparison',
 'tip_labelling',
 'tip_generalisation',
 'tip_duality',
 'tip_reproach',
 'tip_deserves',
 'tip_denial',
 'tip_sacrifice',
 'tip_unavailable',
 'tip_unsolicited',
 'tip_provocation',
 'tip_fact',
 'tip_selfempathy',
 'tip_empathy',
 'tip_dialogue',
 'tip_clarity',
 'tip_activity',
 'tip_demand',
 'tip_person',
 'tip_who',
 'tip_location',
 'tip_where',
 'tip_time',
 'tip_when',
 'tip_action',
 'tip_what',
 'tip_objective',
 'tip_why',
 'tip_thismany',
 'tip_howmany',
 'tip_positive',
 'tip_negative',
 'tip_realistic',
 'tip_unrealistic',
 'tip_feeling',
 'tip_need',
 'group_empathy',
 'group_negative',
 'group_feeling',
 'group_fault',
 'group_unsolicited',
 'group_need',
 'group_realistic',
 'group_precise',
 'group_unprecise',
 'group_unrealistic',
 'grou

In [103]:
#df=df[['Unnamed: 0','id','en','final_vector','rating','label','tips','tips_groups','steps','step_facts','step_emotions','step_needs','step_proposals','tip_morality','tip_positivebias','tip_interpretation','tip_comparison','tip_labelling','tip_generalisation','tip_duality','tip_reproach','tip_deserves','tip_denial','tip_sacrifice','tip_unavailable','tip_unsolicited','tip_provocation','tip_fact','tip_selfempathy','tip_empathy','tip_dialogue','tip_clarity','tip_activity','tip_demand','tip_person','tip_who','tip_location','tip_where','tip_time','tip_when','tip_action','tip_what','tip_objective','tip_why','tip_thismany','tip_howmany','tip_positive','tip_negative','tip_realistic','tip_unrealistic','tip_feeling','tip_need','group_empathy','group_negative','group_feeling','group_fault','group_unsolicited','group_need','group_realistic','group_precise','group_unprecise','group_unrealistic','group_provocation','group_positivebias','group_judgemental','group_demanding','group_fact','group_positive', 'pipeline','Verbal_units','final_verbal_units',]]
#df

In [30]:
df['final_vector'].shape

(11301,)

In [32]:
#convert the vector to numpy array
x=df['final_vector'].to_numpy()
x=x.reshape(-1,1)
x.shape

(11301, 1)

In [33]:
# It is an array of array let's concatenate it one array and rechape it to spacy standard: array of length 300
x1=np.concatenate(np.concatenate(x, axis = 0), axis = 0).reshape(-1, 300) # this "x=x.reshape(-1, 300) produces error
x1.shape

(11301, 300)

In [34]:
x1

array([[-0.19325343,  0.3263942 , -0.00068213, ...,  0.01473643,
        -0.01015704,  0.04242308],
       [-0.08044609,  0.24658439,  0.04340328, ...,  0.03651468,
        -0.12927847,  0.07602477],
       [-0.15276563,  0.24925642, -0.11499526, ..., -0.00363939,
        -0.04042935,  0.04458929],
       ...,
       [-0.14532371,  0.39110047, -0.06625936, ...,  0.07595234,
        -0.04374593,  0.06719923],
       [-0.04292577,  0.26609018, -0.07970441, ...,  0.0573385 ,
        -0.05474704,  0.01911804],
       [-0.15391299,  0.34914994, -0.06995078, ...,  0.06168213,
        -0.072854  , -0.00919154]], dtype=float32)

#Train model on labels: violent / non violent

In [35]:
y = df['label']

In [37]:
x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size = 0.2, random_state = 0, stratify = y)

In [38]:
model = LogisticRegression()
model = model.fit(x_train, y_train)
model_prediction = model.predict(x_test)
print(classification_report(y_test, model_prediction))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86      1193
           1       0.84      0.84      0.84      1068

    accuracy                           0.85      2261
   macro avg       0.85      0.85      0.85      2261
weighted avg       0.85      0.85      0.85      2261



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


#Training on steps

In [39]:
#Eliminate rows where steps sentences with these steps are negligible
steps_minor_list = [
'emotions,facts', 'emotions,needs', 'emotions,proposals',
'facts,emotions,needs','facts,emotions,needs,proposals', 'facts,emotions,proposals',
'facts,emotions,proposals,needs','facts,needs','facts,needs,proposals','facts,proposals,emotions',
'facts,proposals,needs','needs','needs,emotions,facts','needs,proposals','proposals,emotions',
'proposals,facts','proposals,facts,needs','proposals,needs'
]
for i in steps_minor_list:
  df = df[df['steps'] != i]

df

,Unnamed: 0,id,en,rating,label,tips,tips_groups,steps,step_facts,step_emotions,step_needs,step_proposals,tip_morality,tip_positivebias,tip_interpretation,tip_comparison,tip_labelling,tip_generalisation,tip_duality,tip_reproach,tip_deserves,tip_denial,tip_sacrifice,tip_unavailable,tip_unsolicited,tip_provocation,tip_fact,tip_selfempathy,tip_empathy,tip_dialogue,tip_clarity,tip_activity,tip_demand,tip_person,tip_who,tip_location,tip_where,tip_time,tip_when,tip_action,tip_what,tip_objective,tip_why,tip_thismany,tip_howmany,tip_positive,tip_negative,tip_realistic,tip_unrealistic,tip_feeling,tip_need,group_empathy,group_negative,group_feeling,group_fault,group_unsolicited,group_need,group_realistic,group_precise,group_unprecise,group_unrealistic,group_provocation,group_positivebias,group_judgemental,group_demanding,group_fact,group_positive,pipeline,Verbal_units,final_verbal_units,final_vector
0,1,1,sperms and you won,0,0,"comparison,provocation","provocation,judgemental",facts,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,"(sperms, and, you, won)","[you, sperms and won, None]","['you', 'sperms and won', 'None']","[-0.19325343, 0.3263942, -0.00068212557, -0.13..."
1,2,2,a study examined the effects of combined nvc a...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, study, examined, the, effects, of, combine...","[a study, examined , the effects of combined ...","['a study', ' examined ', 'the effects of comb...","[-0.08044609, 0.24658439, 0.04340328, -0.10656..."
2,3,3,a big guy like you needs to stand up for himself,1,0,"labelling,reproach","judgemental,fault",facts,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,"(a, big, guy, like, you, needs, to, stand, up,...","[you, a big guy like needs to stand up for hi...","['you', 'a big guy like needs to stand up for...","[-0.15276563, 0.24925642, -0.11499526, -0.0613..."
3,4,4,a cheetah the fastest land animal can run mile...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, cheetah, the, fastest, land, animal, can, ...","[a cheetah, the fastest land animal can run ,...","['a cheetah', ' the fastest land animal can ru...","[-0.075149335, 0.23446524, -0.032440823, -0.09..."
4,5,5,a combination of words that makes sense is cal...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, combination, of, words, that, makes, sense...","[a combination of words that makes sense, is ...","['a combination of words that makes sense', ' ...","[-0.108618304, 0.38019484, -0.06853195, -0.084..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11294,11295,11293,i will call the authorities if these actions c...,0,0,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(i, will, call, the, authorities, if, these, a...","[i, wll call the authortes f these actons con...","['i', ' wll call the authortes f these actons ...","[-0.120142564, 0.23726754, -0.07398873, -0.089..."
11295,11296,11294,i was worried,4,1,feeling,feeling,emotions,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"(i, was, worried)","[i, was worred, None]","['i', ' was worred', 'None']","[-0.10473377, 0.39818138, -0.01691929, -0.2018..."
11298,11299,11297,i feel frustrated,4,1,feeling,feeling,emotions,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [40]:
# Now we have only the steps of : {'Emotions' , 'Facts', 'Facts Emotions', 'facts,proposals', 'proposals'}
label = {'emotions':0 , 'facts':1 , 'facts emotions':2 , 'facts,proposals':3 , 'proposals':4}
df['numerical_steps'] = (df['steps'].map(label)) 
df

,Unnamed: 0,id,en,rating,label,tips,tips_groups,steps,step_facts,step_emotions,step_needs,step_proposals,tip_morality,tip_positivebias,tip_interpretation,tip_comparison,tip_labelling,tip_generalisation,tip_duality,tip_reproach,tip_deserves,tip_denial,tip_sacrifice,tip_unavailable,tip_unsolicited,tip_provocation,tip_fact,tip_selfempathy,tip_empathy,tip_dialogue,tip_clarity,tip_activity,tip_demand,tip_person,tip_who,tip_location,tip_where,tip_time,tip_when,tip_action,tip_what,tip_objective,tip_why,tip_thismany,tip_howmany,tip_positive,tip_negative,tip_realistic,tip_unrealistic,tip_feeling,tip_need,group_empathy,group_negative,group_feeling,group_fault,group_unsolicited,group_need,group_realistic,group_precise,group_unprecise,group_unrealistic,group_provocation,group_positivebias,group_judgemental,group_demanding,group_fact,group_positive,pipeline,Verbal_units,final_verbal_units,final_vector,numerical_steps
0,1,1,sperms and you won,0,0,"comparison,provocation","provocation,judgemental",facts,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,"(sperms, and, you, won)","[you, sperms and won, None]","['you', 'sperms and won', 'None']","[-0.19325343, 0.3263942, -0.00068212557, -0.13...",1.0
1,2,2,a study examined the effects of combined nvc a...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, study, examined, the, effects, of, combine...","[a study, examined , the effects of combined ...","['a study', ' examined ', 'the effects of comb...","[-0.08044609, 0.24658439, 0.04340328, -0.10656...",1.0
2,3,3,a big guy like you needs to stand up for himself,1,0,"labelling,reproach","judgemental,fault",facts,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,"(a, big, guy, like, you, needs, to, stand, up,...","[you, a big guy like needs to stand up for hi...","['you', 'a big guy like needs to stand up for...","[-0.15276563, 0.24925642, -0.11499526, -0.0613...",1.0
3,4,4,a cheetah the fastest land animal can run mile...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, cheetah, the, fastest, land, animal, can, ...","[a cheetah, the fastest land animal can run ,...","['a cheetah', ' the fastest land animal can ru...","[-0.075149335, 0.23446524, -0.032440823, -0.09...",1.0
4,5,5,a combination of words that makes sense is cal...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, combination, of, words, that, makes, sense...","[a combination of words that makes sense, is ...","['a combination of words that makes sense', ' ...","[-0.108618304, 0.38019484, -0.06853195, -0.084...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11294,11295,11293,i will call the authorities if these actions c...,0,0,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(i, will, call, the, authorities, if, these, a...","[i, wll call the authortes f these actons con...","['i', ' wll call the authortes f these actons ...","[-0.120142564, 0.23726754, -0.07398873, -0.089...",1.0
11295,11296,11294,i was worried,4,1,feeling,feeling,emotions,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"(i, was, worried)","[i, was worred, None]","['i', ' was worred', 'None']","[-0.10473377, 0.39818138, -0.01691929, -0.2018...",0.0
11298,11299,11297,i feel frustrated,4,1,feeling,feeling,emotio

In [41]:
#Beceause this mapping produce float numbers we need to convert it the integers (otherwise later will throw errors)
df1 = df.dropna()
df1['int_numerical_staps'] = df1['numerical_steps'].astype(int)
y1 = df1['int_numerical_staps']
x1 = df1['final_vector']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
x1.shape

(9434,)

In [43]:
#convert the vector to numpy array
x1=df1['final_vector'].to_numpy()
x1=x1.reshape(-1,1)
x1.shape

(9434, 1)

In [44]:
y1.shape

(9434,)

In [45]:
# It is an array of array let's concatenate it one array and rechape it to spacy standard: array of length 300
x2=np.concatenate(np.concatenate(x1, axis = 0), axis = 0).reshape(-1, 300) # this "x1=x1.reshape(-1, 300) produces error
x2.shape

(9434, 300)

In [46]:
x2_train, x2_test, y1_train, y1_test = train_test_split(x2, y1, test_size = 0.2, random_state = 0, stratify = y1)

#Precision on steps
## Reminder: {'emotions':0 , 'facts':1 , 'facts emotions':2 , 'facts,proposals':3 , 'proposals':4}

In [47]:
model1 = LogisticRegression()
model1 = model1.fit(x2_train, y1_train)
model1_prediction = model1.predict(x2_test)
print(classification_report(y1_test, model1_prediction))

              precision    recall  f1-score   support

           0       0.80      0.58      0.67       143
           1       0.81      0.96      0.88      1351
           3       0.68      0.15      0.25       174
           4       0.74      0.47      0.57       219

    accuracy                           0.80      1887
   macro avg       0.76      0.54      0.59      1887
weighted avg       0.79      0.80      0.77      1887



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


#Training on Tips group

In [48]:
mynewlist = set(df['tips_groups'])
print(mynewlist)

{'empathy', 'unrealistic,positive,precise,unsolicited', 'realistic,feeling,positive,precise', 'unsolicited,unprecise,provocation,realistic,positive', 'unsolicited,realistic,unprecise,judgemental,demanding,positive', 'negative,realistic,precise,judgemental,demanding', 'realistic,positive,unprecise,unsolicited', 'unsolicited,realistic,precise,demanding,positive', 'empathy,provocation,fault', 'provocation,feeling', 'fact', 'unrealistic,negative,fault,unprecise', 'provocation', 'empathy,fact,judgemental', 'unrealistic,provocation,positive,unprecise', 'judgemental,positivebias,feeling', 'demanding', 'unsolicited,unprecise,unrealistic,provocation,positive', 'provocation,fault,unsolicited', 'realistic,positive,precise', 'realistic,provocation,positive,unprecise', 'demanding,realistic,unprecise', 'unsolicited,precise,unrealistic,judgemental,positive', 'realistic,positive,fault,unprecise', 'empathy,fact,feeling', 'negative,judgemental', 'unrealistic,negative,unprecise', 'negative,unsolicited,un

In [49]:
#Eliminate rows where steps sentences with these tips are negligible
dft = df
tips_group_minor_list = [
'fault,unsolicited,unprecise,realistic,positive',
'empathy,feeling,precise,realistic,fact,positive', 
'realistic,positive,feeling,unprecise', 'realistic,negative,precise,fault',
'unrealistic,positive,precise,unsolicited', 'realistic,feeling,positive,precise',
'empathy,positivebias,judgemental', 'provocation,judgemental', 
'negative,fault,unsolicited,realistic,unprecise,provocation,judgemental,demanding', 
'demanding,negative,unprecise', 'empathy,provocation,fault,unsolicited', 'negative,fault,unsolicited,unprecise,realistic',
'fault,unsolicited,unprecise,provocation,realistic,positive', 
'negative,unsolicited,precise,provocation,realistic', 'feeling,precise', 
'unsolicited,precise,unrealistic,judgemental,positive', 'empathy,provocation,fault',
'empathy,unsolicited', 'empathy,judgemental,unsolicited', 'realistic,precise,positive,fault', 
'negative,unprecise,unrealistic,provocation,judgemental', 'demanding,judgemental,fault',
'unrealistic,unprecise', 'empathy,positivebias', 
'negative,fault,realistic,precise,demanding', 
'empathy,negative,realistic,unprecise', 
'realistic,provocation,negative,precise', 
'negative,unsolicited,unprecise,provocation,realistic', 
'realistic,negative,unprecise,unsolicited', 'empathy,fact,feeling,judgemental', 
'realistic,empathy,positive,unprecise', 'provocation,fact', 
'unsolicited,realistic,unprecise,provocation,demanding,positive', 
'fault,unsolicited', 'empathy,judgemental', 'negative,unsolicited,realistic,unprecise,demanding', 
'negative,unsolicited,realistic,precise,judgemental,demanding', 'fact,feeling', 
'negative,realistic,precise,judgemental,demanding', 'empathy,realistic,positive,unprecise', 
'unsolicited,realistic,unprecise,demanding,positive', 
'realistic,precise,positive,judgemental', 'negative,fault,unprecise,judgemental,realistic', 
'realistic,empathy,positive,precise', 'precise,unrealistic,provocation,demanding,positive', 
'provocation,unsolicited', 'realistic,fact,positive,unprecise', 'provocation,fault', 
'empathy,fault,unsolicited', 'empathy,positivebias,feeling', 
'empathy,feeling,unprecise,realistic,fact,positive', 'demanding', 
'empathy,positivebias,feeling,judgemental', 'provocation,judgemental,fault,unsolicited', 
'fact,unprecise', 'empathy,demanding', 'negative,unsolicited,realistic,unprecise,provocation,demanding', 
'provocation,judgemental,fault', 'unrealistic,provocation,negative,precise', 
'realistic,unprecise,provocation,judgemental,demanding,positive', 
'negative,fault,unsolicited,precise,judgemental,realistic', 
'demanding,fault', 'judgemental,feeling', 'realistic,positive,precise', 
'unsolicited,realistic,precise,demanding,positive', 'realistic,unprecise', 
'realistic,positive,unprecise', 'realistic,provocation,positive,unprecise', 
'fault,realistic,precise,demanding,positive', 'feeling,precise,realistic,fact,positive', 
'realistic,positive,unprecise,unsolicited', 'empathy,fact,judgemental', 
'judgemental,positivebias,feeling', 'fact,judgemental', 
'negative,realistic,unprecise,judgemental,demanding', 
'realistic,positive,feeling,precise', 'unsolicited,precise,judgemental,realistic,positive', 
'demanding,judgemental', 'empathy,precise', 'judgemental,fault,unsolicited', 
'unsolicited', 'unrealistic,negative,unprecise,unsolicited', 
'realistic,provocation,negative,unprecise', 'unsolicited,unprecise,positivebias,realistic,positive', 
'feeling,fault', 'realistic,negative,unprecise', 'empathy,provocation', 
'realistic,positive,fault,unprecise', 'realistic,positivebias,positive,precise', 
'provocation,fault,unprecise', 'judgemental,fault', 'empathy,judgemental,fault', 
'precise,judgemental,realistic,fact,positive', 'provocation,feeling,judgemental,fault', 
'empathy,precise,positivebias,realistic,positive', 'empathy,realistic,positive,precise', 
'negative,realistic,unprecise,provocation,demanding', 
'unrealistic,provocation,negative,unprecise', 'feeling,unsolicited', 
'realistic,negative,fault,unprecise', 'feeling,judgemental,fault', 
'unsolicited,unprecise,provocation,realistic,positive', 
'negative,unprecise,provocation,judgemental,realistic', 
'empathy,provocation,judgemental', 'realistic,negative,precise,unsolicited', 
'provocation,fault,unsolicited', 'positivebias,need,feeling', 
'fault,unsolicited,precise,realistic,positive', 
'negative,fault,realistic,unprecise,demanding', 
'fault,unsolicited,precise,judgemental,realistic,positive', 
'realistic,positive,judgemental,precise', 'unsolicited,precise,provocation,realistic,positive', 
'realistic,unprecise,provocation,demanding,positive', 
'negative,fault,unprecise,unrealistic,provocation,demanding', 
'unsolicited,unprecise,judgemental,realistic,positive', 
'realistic,positive,judgemental,unprecise', 'demanding,unrealistic,positive,unprecise', 
'judgemental,unsolicited', 'fault,precise,provocation,realistic,positive', 
'empathy,unsolicited,unprecise,realistic,positive', 'empathy,provocation,unsolicited', 
'empathy,precise,judgemental,realistic,fact,positive', 'realistic,precise', 
'unrealistic,negative,unprecise', 'unsolicited,unprecise,provocation,judgemental,realistic,positive', 
'empathy,fact', 'fault,unsolicited,precise,provocation,realistic,positive', 
'negative,unsolicited,unprecise,unrealistic,provocation', 'empathy,fault', 
'unrealistic,positive,unprecise,unsolicited', 'negative,unprecise,unrealistic,provocation,judgemental,demanding', 
'realistic,precise,provocation,demanding,positive', 'demanding,realistic,unprecise', 
'empathy,need', 'realistic,negative,precise', 'negative,realistic,precise,provocation,demanding', 
'feeling,unprecise,realistic,fact,positive', 'empathy,judgemental,fault,unsolicited', 
'judgemental,feeling,fault', 'positivebias,feeling', 'empathy,feeling,judgemental', 
'negative,judgemental', 'provocation', 'demanding,negative,realistic,precise', 
'feeling,judgemental', 'unrealistic,provocation,positive,unprecise', 
'unrealistic,positive,unprecise', 'positivebias,judgemental', 
'negative,unsolicited,unprecise,judgemental,realistic', 'empathy,feeling', 
'unrealistic,negative,precise,unsolicited', 'realistic,positivebias,positive,unprecise', 
'unprecise', 'realistic,unprecise,judgemental,demanding,positive', 
'unprecise,provocation,judgemental,realistic,positive', 'provocation,judgemental,unsolicited', 
'negative,fault,unsolicited,precise,realistic', 'unsolicited,realistic,unprecise,judgemental,demanding,positive', 
'precise', 'provocation,feeling', 'fact,fault', 'empathy,fact,feeling', 
'unsolicited,unprecise,unrealistic,provocation,positive', 'demanding,realistic,positive,unprecise', 
'negative,unsolicited,precise,unrealistic,provocation', 'negative,unprecise,unrealistic,judgemental,demanding', 
'unrealistic,negative,fault,unprecise', 'demanding,negative,realistic,unprecise', 
'demanding,realistic,precise', 'negative,unsolicited,realistic,precise,demanding']


#tips_list =['empathy', 'fact', 'fault','feeling', 'judgemental', 'provocation', 'positivebias']
for i in tips_group_minor_list:
  dft = dft[dft['tips_groups'] != i ]
dft

,Unnamed: 0,id,en,rating,label,tips,tips_groups,steps,step_facts,step_emotions,step_needs,step_proposals,tip_morality,tip_positivebias,tip_interpretation,tip_comparison,tip_labelling,tip_generalisation,tip_duality,tip_reproach,tip_deserves,tip_denial,tip_sacrifice,tip_unavailable,tip_unsolicited,tip_provocation,tip_fact,tip_selfempathy,tip_empathy,tip_dialogue,tip_clarity,tip_activity,tip_demand,tip_person,tip_who,tip_location,tip_where,tip_time,tip_when,tip_action,tip_what,tip_objective,tip_why,tip_thismany,tip_howmany,tip_positive,tip_negative,tip_realistic,tip_unrealistic,tip_feeling,tip_need,group_empathy,group_negative,group_feeling,group_fault,group_unsolicited,group_need,group_realistic,group_precise,group_unprecise,group_unrealistic,group_provocation,group_positivebias,group_judgemental,group_demanding,group_fact,group_positive,pipeline,Verbal_units,final_verbal_units,final_vector,numerical_steps
1,2,2,a study examined the effects of combined nvc a...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, study, examined, the, effects, of, combine...","[a study, examined , the effects of combined ...","['a study', ' examined ', 'the effects of comb...","[-0.08044609, 0.24658439, 0.04340328, -0.10656...",1.0
3,4,4,a cheetah the fastest land animal can run mile...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, cheetah, the, fastest, land, animal, can, ...","[a cheetah, the fastest land animal can run ,...","['a cheetah', ' the fastest land animal can ru...","[-0.075149335, 0.23446524, -0.032440823, -0.09...",1.0
4,5,5,a combination of words that makes sense is cal...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, combination, of, words, that, makes, sense...","[a combination of words that makes sense, is ...","['a combination of words that makes sense', ' ...","[-0.108618304, 0.38019484, -0.06853195, -0.084...",1.0
6,7,7,a group of words which makes a complete sense ...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, group, of, words, which, makes, a, complet...",[a group of words which makes a complete sense...,['a group of words which makes a complete sens...,"[-0.0831828, 0.27531555, -0.08243834, 0.008643...",1.0
9,10,10,a nurse is ten times more likely to have chron...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, nurse, is, ten, times, more, likely, to, h...","[a nurse, is ten times more likely to have , ...","['a nurse', ' is ten times more likely to have...","[-0.16795439, 0.29400578, -0.050677855, -0.011...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11292,11293,11291,i felt angry because i need my living room to ...,4,1,feeling,feeling,emotions,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"(i, felt, angry, because, i, need, my, living,...","[i, felt angry because need my lvng room to ...","['i', ' felt angry because need my lvng room ...","[-0.07443942, 0.260743, -0.108790495, -0.10957...",0.0
11294,11295,11293,i will call the authorities if these actions c...,0,0,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(i, will, call, the, authorities, if, these, a...","[i, wll call the authortes f these actons con...","['i', ' wll call the authortes f

In [51]:
dft

,Unnamed: 0,id,en,rating,label,tips,tips_groups,steps,step_facts,step_emotions,step_needs,step_proposals,tip_morality,tip_positivebias,tip_interpretation,tip_comparison,tip_labelling,tip_generalisation,tip_duality,tip_reproach,tip_deserves,tip_denial,tip_sacrifice,tip_unavailable,tip_unsolicited,tip_provocation,tip_fact,tip_selfempathy,tip_empathy,tip_dialogue,tip_clarity,tip_activity,tip_demand,tip_person,tip_who,tip_location,tip_where,tip_time,tip_when,tip_action,tip_what,tip_objective,tip_why,tip_thismany,tip_howmany,tip_positive,tip_negative,tip_realistic,tip_unrealistic,tip_feeling,tip_need,group_empathy,group_negative,group_feeling,group_fault,group_unsolicited,group_need,group_realistic,group_precise,group_unprecise,group_unrealistic,group_provocation,group_positivebias,group_judgemental,group_demanding,group_fact,group_positive,pipeline,Verbal_units,final_verbal_units,final_vector,numerical_steps
1,2,2,a study examined the effects of combined nvc a...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, study, examined, the, effects, of, combine...","[a study, examined , the effects of combined ...","['a study', ' examined ', 'the effects of comb...","[-0.08044609, 0.24658439, 0.04340328, -0.10656...",1.0
3,4,4,a cheetah the fastest land animal can run mile...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, cheetah, the, fastest, land, animal, can, ...","[a cheetah, the fastest land animal can run ,...","['a cheetah', ' the fastest land animal can ru...","[-0.075149335, 0.23446524, -0.032440823, -0.09...",1.0
4,5,5,a combination of words that makes sense is cal...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, combination, of, words, that, makes, sense...","[a combination of words that makes sense, is ...","['a combination of words that makes sense', ' ...","[-0.108618304, 0.38019484, -0.06853195, -0.084...",1.0
6,7,7,a group of words which makes a complete sense ...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, group, of, words, which, makes, a, complet...",[a group of words which makes a complete sense...,['a group of words which makes a complete sens...,"[-0.0831828, 0.27531555, -0.08243834, 0.008643...",1.0
9,10,10,a nurse is ten times more likely to have chron...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, nurse, is, ten, times, more, likely, to, h...","[a nurse, is ten times more likely to have , ...","['a nurse', ' is ten times more likely to have...","[-0.16795439, 0.29400578, -0.050677855, -0.011...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11292,11293,11291,i felt angry because i need my living room to ...,4,1,feeling,feeling,emotions,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"(i, felt, angry, because, i, need, my, living,...","[i, felt angry because need my lvng room to ...","['i', ' felt angry because need my lvng room ...","[-0.07443942, 0.260743, -0.108790495, -0.10957...",0.0
11294,11295,11293,i will call the authorities if these actions c...,0,0,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(i, will, call, the, authorities, if, these, a...","[i, wll call the authortes f these actons con...","['i', ' wll call the authortes f

In [52]:
# Now we have only the tips_groups of : {'empathy' , 'fact', 'fault', 'judgemental', 'positivebias', 'feeling'}
label = {'empathy':0 , 'fact':1 , 'fault':2 , 'judgemental':3 , 'positivebias':4 , 'feeling':5 }
dft['numerical_tips'] = (dft['tips_groups'].map(label)) 
dft

,Unnamed: 0,id,en,rating,label,tips,tips_groups,steps,step_facts,step_emotions,step_needs,step_proposals,tip_morality,tip_positivebias,tip_interpretation,tip_comparison,tip_labelling,tip_generalisation,tip_duality,tip_reproach,tip_deserves,tip_denial,tip_sacrifice,tip_unavailable,tip_unsolicited,tip_provocation,tip_fact,tip_selfempathy,tip_empathy,tip_dialogue,tip_clarity,tip_activity,tip_demand,tip_person,tip_who,tip_location,tip_where,tip_time,tip_when,tip_action,tip_what,tip_objective,tip_why,tip_thismany,tip_howmany,tip_positive,tip_negative,tip_realistic,tip_unrealistic,tip_feeling,tip_need,group_empathy,group_negative,group_feeling,group_fault,group_unsolicited,group_need,group_realistic,group_precise,group_unprecise,group_unrealistic,group_provocation,group_positivebias,group_judgemental,group_demanding,group_fact,group_positive,pipeline,Verbal_units,final_verbal_units,final_vector,numerical_steps,numerical_tips
1,2,2,a study examined the effects of combined nvc a...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, study, examined, the, effects, of, combine...","[a study, examined , the effects of combined ...","['a study', ' examined ', 'the effects of comb...","[-0.08044609, 0.24658439, 0.04340328, -0.10656...",1.0,1
3,4,4,a cheetah the fastest land animal can run mile...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, cheetah, the, fastest, land, animal, can, ...","[a cheetah, the fastest land animal can run ,...","['a cheetah', ' the fastest land animal can ru...","[-0.075149335, 0.23446524, -0.032440823, -0.09...",1.0,1
4,5,5,a combination of words that makes sense is cal...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, combination, of, words, that, makes, sense...","[a combination of words that makes sense, is ...","['a combination of words that makes sense', ' ...","[-0.108618304, 0.38019484, -0.06853195, -0.084...",1.0,1
6,7,7,a group of words which makes a complete sense ...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, group, of, words, which, makes, a, complet...",[a group of words which makes a complete sense...,['a group of words which makes a complete sens...,"[-0.0831828, 0.27531555, -0.08243834, 0.008643...",1.0,1
9,10,10,a nurse is ten times more likely to have chron...,4,1,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(a, nurse, is, ten, times, more, likely, to, h...","[a nurse, is ten times more likely to have , ...","['a nurse', ' is ten times more likely to have...","[-0.16795439, 0.29400578, -0.050677855, -0.011...",1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11292,11293,11291,i felt angry because i need my living room to ...,4,1,feeling,feeling,emotions,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"(i, felt, angry, because, i, need, my, living,...","[i, felt angry because need my lvng room to ...","['i', ' felt angry because need my lvng room ...","[-0.07443942, 0.260743, -0.108790495, -0.10957...",0.0,5
11294,11295,11293,i will call the authorities if these actions c...,0,0,fact,fact,facts,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(i, will, call, the, authorities, if, these, a...","[i, wll call the authortes f these actons con...","[

In [55]:
dft2 = dft.dropna()
dft2['int_numerical_tips'] = dft2['numerical_tips'].astype(int)
yt2 = dft2['int_numerical_tips']
xt2 = dft2['final_vector']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [56]:
xt2.shape

(5774,)

In [58]:
#convert the vector to numpy array
xt2=dft2['final_vector'].to_numpy()
xt2=xt2.reshape(-1,1)
xt2.shape

(5774, 1)

In [59]:
yt2.shape

(5774,)

In [60]:
# It is an array of array let's concatenate it one array and rechape it to spacy standard: array of length 300
xt3=np.concatenate(np.concatenate(xt2, axis = 0), axis = 0).reshape(-1, 300) # this "x1=x1.reshape(-1, 300) produces error
xt3.shape

(5774, 300)

In [61]:
xt3_train, xt3_test, yt2_train, yt2_test = train_test_split(xt3, yt2, test_size = 0.2, random_state = 0, stratify = yt2)

#Precision on tips_group
## Reminder: {'empathy':0 , 'fact':1 , 'fault':2 , 'judgemental':3 , 'positivebias':4 , 'feeling':5 }

In [62]:
modelf = LogisticRegression()
modelf0 = modelf.fit(xt3_train, yt2_train)
modelf0_prediction = modelf0.predict(xt3_test)
print(classification_report(yt2_test, modelf0_prediction))

              precision    recall  f1-score   support

           0       0.74      0.75      0.74       198
           1       0.74      0.78      0.76       263
           2       0.53      0.23      0.32       134
           3       0.72      0.85      0.78       409
           4       1.00      0.33      0.49        40
           5       0.69      0.74      0.72       111

    accuracy                           0.72      1155
   macro avg       0.74      0.61      0.63      1155
weighted avg       0.71      0.72      0.70      1155



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


#Saving final model

In [64]:
sereniiti_wf = "sereniiti_w.pkl" 

In [65]:
pickle.dump(model, open('sereniiti_wf', 'wb'))

In [66]:
model = pickle.load(open('sereniiti_wf', 'rb'))

In [67]:
model

LogisticRegression()

In [68]:
model_prediction = model.predict(x_test)

In [69]:
ex='you are really a $%bad guy'
ex=get_clean(ex)
vec=sent_vector(ex)
model.predict(vec.reshape(1, -1))

array([0])

In [70]:
ex1='oooh shit'
ex1=get_clean(ex1)
vec=sent_vector(ex1)
model.predict(vec.reshape(1, -1))

array([0])

In [71]:
ex1='I like that'
ex1=get_clean(ex1)
vec=sent_vector(ex1)
model.predict(vec.reshape(1, -1))

array([1])

# Model Evaluation:Metrics

In [73]:
# Mean Square Error
print("MSE of violent/non violent model :" ,mean_squared_error(y_test, model_prediction))
print("MSE of trained teps model :" ,mean_squared_error(y1_test, model1_prediction))
print("MSE of trained on tips-group model :" ,mean_squared_error(yt2_test, modelf0_prediction))

MSE of violent/non violent model : 0.15081822202565237
MSE of trained teps model : 1.0196078431372548
MSE of trained on tips-group model : 1.3731601731601732
